# Web Crawler
---

### Imports

In [1]:
import re
import requests
from collections import deque

from bs4 import BeautifulSoup as bs

import urllib.request
import urllib.parse as parse
import urllib.robotparser as rp

from socket import gethostbyname, gaierror

import time

import pandas as pd

## Initialisation

We set an initial URL in accordance with our theme, a maximum number of links to save and a maximum number of links to download the contents of. This way, if the links we get do not want to be crawled, we do not need to go back and save more links. 

In [2]:
URL = 'https://en.wikipedia.org/wiki/Chocolate'
max_links = 50     # max nb of links to crawl
max_cont = 20      # max nb of links to get content of 

## Getting the links

We use one function to retrieve all external links in the page (links to the same domain do not begin by htts, and as such are not spotted by Beautiful Soup).

In [3]:
# Use regex to extract all links from url, stop if we reach maximum
def get_all_links (url, counter, Links = [], max_l=max_links):
    
    html_page = urllib.request.urlopen(url)   # Open url on internet
    soup = bs(html_page, 'lxml')              # Open contents of url
    
    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        if(counter < max_l)&(link not in Links):
            Links.append(link.get('href'))
            counter += 1
        else:
            break
            
    return (Links, counter)

## Crawling without considering robots.txt

This method return at most max_c links and saves at most max_c, starting from an initial url and getting its children links, their children links and so on in a breadth-first fashion.   

Returns:
    list - a list of strings that are the links

In [4]:
"""This method return at most max_l links, starting from an initial url and getting its children links, their children links and so on in a breadth-first fashion.

Returns:
    list
        a list of strings that are the links
"""

def crawl_for_links(Links=[], pointer=0, max_l=max_links):
    
    url = Links[pointer]
    counter = len(Links)
        
    Links, counter = get_all_links (url, counter, Links, max_l)
    
    # If we still do not have enough links: repeat the process with the next url from the list
    if counter < max_l :
        Links = crawl_for_links(Links, pointer+1, max_l - counter)
        

    return Links

In [5]:
links_crawled = crawl_for_links([URL])
print(links_crawled)
print(len(links_crawled))

['https://en.wikipedia.org/wiki/Chocolate', 'http://ndb.nal.usda.gov/ndb/foods/show/6153?fgcd=&manu=&lfacet=&format=&count=&max=35&offset=&sort=&qlookup=Candies%2C+milk+chocolate', 'http://www.eluniversal.com.mx/notas/526113.html', 'http://archive.fieldmuseum.org/Chocolate/history_mesoamerican7.html', 'http://www.bartleby.com/61/68/C0316800.html', 'http://antiquity.ac.uk/projgall/powis/index.html', 'http://news.sciencemag.org/2013/01/earliest-evidence-chocolate-north-america', 'http://www.museum.upenn.edu/new/news/fullrelease.php?which=306', 'http://archive.fieldmuseum.org/Chocolate/history_mesoamerican5.html', 'http://archive.fieldmuseum.org/Chocolate/history_mesoamerican3.html', 'http://archive.fieldmuseum.org/Chocolate/history_mesoamerican4.html', 'http://archive.fieldmuseum.org/chocolate/history.html', 'http://findarticles.com/p/articles/mi_m1310/is_1990_Jan/ai_8560999', 'http://www.newyorker.com/reporting/2007/10/29/071029fa_fact_buford', 'http://www.exploratorium.edu/exploring/ex

In [6]:
# Use regex to extract all links from url, stop if we reach maximum
def get_all_links_robot(url, counter, Links = [], max_l=max_links):
    
    html_page = urllib.request.urlopen(url)   # Open url on internet
    soup = bs(html_page, 'lxml')              # Open contents of url
    
    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        print("Counter = {} - max_l = {}".format(counter, max_l))
        if(counter < max_l)&(link not in Links):
            save_link_robot(link.get('href'), str(counter))
            Links.append(link.get('href'))
            counter += 1
        else:
            break
            
    return (Links, counter)

## Saving the contents

This function saves the link under a specified filename. The path for that file can be changed in the code.

In [7]:
def save_link(url, filename):
    doc = requests.get(url)
    
    # Put here the path where you want to save the link's contents
    name = "Docs_crawled_noRobot/" + filename + ".html"
    
    with open(name, 'wb') as fOut:
        fOut.write(doc.content)

In [8]:
def save_link_robot(url, filename):
    try:
        doc = requests.get(url.strip())
        # Put here the path where you want to save the link's contents
        name = "docs_crawled/" + filename + ".html"
        with open(name, 'wb') as fOut:
            fOut.write(doc.content)
    except gaierror:
        print("Link not reachable")
    except requests.exceptions.ConnectionError:
        print("Connection Error")

In [9]:
# Test
save_link (URL, '000')
filename = 3*'0' + '1'
filename

'0001'

We chose to have each file's name be its index in the list of links, on 4 characters (for instance: 0008 for the 9th link).

In [10]:
# Prepare new filename
def new_filename (count, fname = '0000', conv = [3, 1]):
    nb_zeros = conv[0]
    pw = conv[1]
    
    if (count > 10**pw):
        nb_zeros -= 1
        pw += 1 
    # If the file index is above the next power of 10, we add one less 0
    # to the filename than we did before
    filename = nb_zeros*'0' + str (count)
    
    return (filename, conv)

In [11]:
filename = "0000"
nb_zeros = 3
pw = 1

conventions = [nb_zeros, pw]

count = 0

for url in links_crawled:
    # Save the link
    save_link(url, filename)
    count += 1
    # Break when we have enough links
    if count >= max_cont : break
        
    filename, conventions = new_filename(count, filename, conventions)

## Handling robots.txt permissions and crawler delays

We save lists of domains, robots.txt urls, crawler delays and times when each url's domain was last crawled. The information on the same url will be at the same index i in all lists (i.e. in line i of the matrix D). We then use these to wait the correct amount of time before crawling each domain a second time. 

In [12]:
domains = []
robots = []
delays = []
time_crawled = []

Links = [URL]

In [13]:
# Method with D as list of lists (domains, robots, delay, time_crawled)

def Crawler (D = [domains, robots, delays, time_crawled], L=Links, pointer=0, \
             fname = '0000', max_l=max_links, max_c = max_cont, conv = [3, 1]):
    
    counter = len(L)
    #print("Nb links in Links: ", counter)
    #print("Pointer: ", pointer)
    already_crawled = True
    
    # Issue with some links: don't parse them        
    #if not (pointer in [1, 4]):
    
    url = L[pointer]
    print(url)

    # Get url domain
    URL_parsed = parse.urlparse(url)
    dom = URL_parsed.netloc
    #print("Domain: ", dom)

    # If we have not already tried to crawl in the domain
    if not (dom in domains):

        #print("New domain")
        already_crawled = False        
        # Get robots.txt link
        r = URL_parsed.scheme + "://" + dom + "/robots.txt"
        #print("Robots.txt link: ", r)

        domains.append(dom)
        robots.append(r)
        delays.append(20)
        time_crawled.append(-1)


    i = domains.index(dom)
    #print("Domain index: ", i)
    r = robots[i]
    #print("Robots.txt link: ", r)

    # If our domain is not dead (xocoatl refuses all robots)
    if (not dom in ["www.cfsan.fda.gov", "www.cocoatree.org", "nca.files.cms-plus.com", \
                    "www.xocoatl.org", "www.cfsan.fda.gov"]):
        
        # Access robots.txt
        RP = rp.RobotFileParser()         
        RP.set_url(r) 
        RP.read()

        # If the robot parser correctly opens the robots.txt. 
        # This test is actually too harsh, but it's better to not parse some links that could be saved rather than 
        # stop the whole process (some robots.txt links actually send us directly to the home domain first, or don't exist)
        if (len(RP.entries) != 0):

            # Get crawl delay (if it is given and we don't already have it)
            if ((not already_crawled) & (RP.crawl_delay("*") != None)):
                delays.pop(i) 
                delays.insert(i, RP.crawl_delay("*"))

            # If we can crawl the file
            if (RP.can_fetch("*", url)):

                # Wait appropriate time if we have already crawled the domain
                if (already_crawled):
                    wait = max(0, delays[i] - (time.time() - time_crawled[i]))
                    if (wait > 0): 
                               time.sleep(wait)

                
                max_c -= 1
                print("\n MAX_C - > {}".format(max_c))
                print("We are saving the contents of ", url)

                # Update time we last crawled the domain
                time_crawled.pop(i)
                time_crawled.insert(i, time.time())                  

                # Prepare filename for next page
                fname, conv = new_filename(pointer + 1, fname, conv)

                # If we do not have enough links, save those referenced in the page
                if (len(L) < max_l):
                    L, counter = get_all_links_robot(L[pointer], counter, L, max_l)
    print(D)
    #print(L)
    
    # If we have not saved enough links: repeat
    if (counter < max_l):
        pointer += 1
        print("COUNTER_IF = {} - max_l = {}, pointer = {}".format(counter, max_l, pointer))
        D, L = Crawler (D, L, pointer, fname, max_l, max_c, conv)
    else:
        return (D, L)
    
    return (D, L)  

In [14]:
# Emptying lists from previous tests
for l in [domains, robots, delays, time_crawled]:
    while len(l) > 0:
            l.pop(0)

while (len(Links) > 1):
    Links.pop(1)

# Save contents of the root link
save_link_robot(URL, "000")
D, Links = Crawler()

https://en.wikipedia.org/wiki/Chocolate

 MAX_C - > 19
We are saving the contents of  https://en.wikipedia.org/wiki/Chocolate
Counter = 1 - max_l = 50
Counter = 2 - max_l = 50
Counter = 3 - max_l = 50
Counter = 4 - max_l = 50
Counter = 5 - max_l = 50
Counter = 6 - max_l = 50
Counter = 7 - max_l = 50
Counter = 8 - max_l = 50
Counter = 9 - max_l = 50
Counter = 10 - max_l = 50
Counter = 11 - max_l = 50
Counter = 12 - max_l = 50
Counter = 13 - max_l = 50
Counter = 14 - max_l = 50
Counter = 15 - max_l = 50
Counter = 16 - max_l = 50
Counter = 17 - max_l = 50
Counter = 18 - max_l = 50
Counter = 19 - max_l = 50
Counter = 20 - max_l = 50
Counter = 21 - max_l = 50
Connection Error
Counter = 22 - max_l = 50
Counter = 23 - max_l = 50
Counter = 24 - max_l = 50
Counter = 25 - max_l = 50
Counter = 26 - max_l = 50
Counter = 27 - max_l = 50
Counter = 28 - max_l = 50
Counter = 29 - max_l = 50
Counter = 30 - max_l = 50
Counter = 31 - max_l = 50
Counter = 32 - max_l = 50
Counter = 33 - max_l = 50
Counter 